In [1135]:
import nltk
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.feature_selection import RFE, RFECV, f_regression, chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV, KFold, RandomizedSearchCV, HalvingGridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.preprocessing import PowerTransformer, QuantileTransformer
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.neighbors import  KNeighborsClassifier
from datetime import datetime as dt
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from imblearn.combine import SMOTETomek
from collections import Counter
from mlxtend.feature_selection import ColumnSelector
import warnings
# Settings the warnings to be ignored
warnings.filterwarnings('ignore')

In [1136]:
# Displaying all columns
pd.set_option('display.max_columns', None)

# #Display all data in columns
# pd.set_option("display.max_colwidth", None)

# np.set_printoptions(threshold=None, edgeitems=None, linewidth=None, suppress=None)
# pd.set_option("display.max_rows", None)
# pd.set_option("display.expand_frame_repr", True)
# pd.set_option('display.width', 1000)


In [1137]:
#Storing Accuracy

data_accuracy = pd.DataFrame(columns=['Algo', 'Accuracy(KFold)', 'Test', 'Train', 'Recall', 'Precision', 'F1', 'ROC_AUC_SCORE'])
def store(data):
    data_accuracy.loc[len(data_accuracy.index)] = data
    data_accuracy.to_csv('loan_prediction_metric_scores.csv')
    print(data_accuracy)



def get_info_pipeline(pipe_name):
    sub_pipe = pipe_name[:-1]
    transformed_df = pd.DataFrame(
        data=sub_pipe.fit_transform(X_train, y_train),
        columns=sub_pipe.get_feature_names_out(),
    )
    print(transformed_df)

In [1138]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [1139]:
df1 = pd.read_csv('tmdb_5000_credits.csv')
df2 = pd.read_csv('tmdb_5000_movies.csv')

In [1140]:
df1.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [1141]:
#Renaming the 'movie_id' col in df1 to 'id' of df2 

In [1142]:
df1.rename(columns = {'movie_id':'id'}, inplace = True)

In [1143]:
df1.head(1)

,id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [1144]:
#joining the credits amd movie dataframes

In [1145]:
df2 = pd.merge(df1, df2, on='id')

In [1146]:
df2.head(1)

,id,title_x,cast,crew,budget,genres,homepage,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title_y,vote_average,vote_count
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [1147]:
#Drpping columns

In [1148]:
df2.columns

Index(['id', 'title_x', 'cast', 'crew', 'budget', 'genres', 'homepage',
       'keywords', 'original_language', 'original_title', 'overview',
       'popularity', 'production_companies', 'production_countries',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title_y', 'vote_average', 'vote_count'],
      dtype='object')

In [1149]:
df2.drop(columns=['status', 'runtime'], axis=1, inplace=True)

In [1150]:
df2.drop(columns=['title_x'], axis=1, inplace=True)

In [1151]:
df2.head(1)

,id,cast,crew,budget,genres,homepage,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,spoken_languages,tagline,title_y,vote_average,vote_count
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Enter the World of Pandora.,Avatar,7.2,11800


In [1152]:
df2.drop(columns=['title_y'], axis=1, inplace=True)

In [1153]:
df2.shape

(4803, 19)

In [1154]:
df2.head(1)

,id,cast,crew,budget,genres,homepage,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,spoken_languages,tagline,vote_average,vote_count
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Enter the World of Pandora.,7.2,11800


In [1155]:
df2.drop(columns=['revenue', 'release_date'], axis=1, inplace=True)

In [1156]:
df2.drop(columns=['budget', 'homepage', 'popularity', 'spoken_languages', 'production_countries', 'vote_average', 'vote_count'], axis=1, inplace=True)

In [1157]:
df2.shape

(4803, 10)

In [1158]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview,production_companies,tagline
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...",Enter the World of Pandora.


In [1159]:
df2.isnull().sum()

id                        0
cast                      0
crew                      0
genres                    0
keywords                  0
original_language         0
original_title            0
overview                  3
production_companies      0
tagline                 844
dtype: int64

In [1160]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    4803 non-null   int64 
 1   cast                  4803 non-null   object
 2   crew                  4803 non-null   object
 3   genres                4803 non-null   object
 4   keywords              4803 non-null   object
 5   original_language     4803 non-null   object
 6   original_title        4803 non-null   object
 7   overview              4800 non-null   object
 8   production_companies  4803 non-null   object
 9   tagline               3959 non-null   object
dtypes: int64(1), object(9)
memory usage: 412.8+ KB


In [1161]:
df2.drop(columns='production_companies', axis=1, inplace=True)

In [1162]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview,tagline
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.


In [1163]:
df2.shape

(4803, 9)

In [1164]:
#Dropping null rows

In [1165]:
df2.isnull().sum()

id                     0
cast                   0
crew                   0
genres                 0
keywords               0
original_language      0
original_title         0
overview               3
tagline              844
dtype: int64

In [1166]:
df2.drop(columns='tagline', axis=1, inplace=True)

In [1167]:
df2.dropna(inplace=True)

In [1168]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [1169]:
df2.isnull().sum()

id                   0
cast                 0
crew                 0
genres               0
keywords             0
original_language    0
original_title       0
overview             0
dtype: int64

In [1170]:
#Preprocessing

In [1171]:
##Genre

In [1172]:
df2.iloc[0, 3]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [1173]:
import ast

In [1174]:
def fetch_genres(string):
    ls = []
    for i in  ast.literal_eval(string):
        ls.append(i['name'])
    return ls

In [1175]:
df2['genres'] = df2['genres'].apply(fetch_genres)

In [1176]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [1177]:
df2.iloc[3, 3]

['Action', 'Crime', 'Drama', 'Thriller']

In [1178]:
##Crew

In [1179]:
def fetch_dir_and_prod(string):
    ls = []
    for i in  ast.literal_eval(string):
        if i['job']=='Director':
            ls.append(i['name'])
            break
    return ls

In [1180]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [1181]:
msg = " "
msg = df2.loc[4].crew

In [1182]:
msg

'[{"credit_id": "52fe479ac3a36847f813eaa3", "department": "Writing", "gender": 2, "id": 7, "job": "Screenplay", "name": "Andrew Stanton"}, {"credit_id": "52fe479ac3a36847f813ea65", "department": "Directing", "gender": 2, "id": 7, "job": "Director", "name": "Andrew Stanton"}, {"credit_id": "58c69e819251411d30017673", "department": "Crew", "gender": 2, "id": 7879, "job": "Thanks", "name": "John Lasseter"}, {"credit_id": "52fe479ac3a36847f813eac5", "department": "Production", "gender": 2, "id": 490, "job": "Producer", "name": "Colin Wilson"}, {"credit_id": "58c68f2b9251411d30016cf2", "department": "Production", "gender": 2, "id": 2073, "job": "Casting", "name": "Gail Stevens"}, {"credit_id": "58c68ef19251411d62016ec3", "department": "Sound", "gender": 2, "id": 3996, "job": "Sound Re-Recording Mixer", "name": "Tom Johnson"}, {"credit_id": "52fe479ac3a36847f813ead7", "department": "Editing", "gender": 2, "id": 6958, "job": "Editor", "name": "Eric Zumbrunnen"}, {"credit_id": "58c69a9ec3a3684

In [1183]:
import ast
msg = ast.literal_eval(msg)

In [1184]:
for i in msg:
    if i['job']=='Director':
        print(i['name'])
        break

Andrew Stanton


In [1185]:
df2['crew'] = df2['crew'].apply(fetch_dir_and_prod)

In [1186]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [1187]:
##keywords

In [1188]:
msg = df2.iloc[0, 4]

In [1189]:
msg

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [1190]:
def fetch_keyowrds(string):
    ls = []
    for i in  ast.literal_eval(string):
        ls.append(i['name'])
            
    return ls

In [1191]:
fetch_keyowrds(msg)

['culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d']

In [1192]:
df2['keywords'] = df2['keywords'].apply(fetch_genres)

In [1193]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [1194]:
##cast (first 3)

In [1195]:
msg = df2.iloc[0, 1]

In [1196]:
msg

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [1197]:
def fetch_crew_first_three(string):
    ls = []
    cnt = 3
    for i in  ast.literal_eval(string):
        if cnt==0:
            break
        else:
            ls.append(i['name'])
        cnt=cnt-1
    return ls

In [1198]:
msg

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [1199]:
fetch_crew_first_three(msg)

['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver']

In [1200]:
df2['cast'] = df2['cast'].apply(fetch_crew_first_three)

In [1201]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview
0,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [1202]:
df2.rename(columns = {'crew':'director'}, inplace = True)

In [1203]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview
0,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [1204]:
#Preprocessing 'overview' column

#Lowercasing

df2['overview'] = df2['overview'].str.lower()


# Removing HTML tags

import re

def remove_html(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)

df2['overview'] = df2['overview'].apply(remove_html)
# print(df.head())


#Remove @

def remove_at_the_rate(text):

  ls = []
  new = []

  ls = nlp(text)

  for word in ls:
        if word.text != "@":
            new.append(word.text)

  return ' '.join(new)

df2['overview'] = df2['overview'].apply(remove_at_the_rate)



#Removing URL

import re

def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

df2['overview'] = df2['overview'].apply(remove_url)
# print(df.head())


#Remmove punctuation

import string

punc = string.punctuation

def  remove_punc(text):

    return text.translate(str.maketrans('', '', punc))

df2['overview'] = df2['overview'].apply(remove_punc)


#Correcting spelling

# from textblob import TextBlob

# def spell_check(text):

#     textblb = TextBlob(text)
#     return textblb.correct().string

# X['remainder__text'] = X['remainder__text'].apply(spell_check)


from autocorrect import Speller

check = Speller()

def check_spell(text):

  return check(text)

df2['overview'] = df2['overview'].apply(check_spell)


# Removing stop words


from nltk.corpus import stopwords

stopwords = stopwords.words('english')

def remove_stop_words(text):
    ls = []
    new = []

    ls = nlp(text)

    for word in ls:
        if word.text not in stopwords:

            new.append(word.text)

    return ' '.join(new)

df2['overview'] = df2['overview'].apply(remove_stop_words)


#Removing Contradictions

import contractions

def remove_contradictions(text):

  return " ".join([contractions.fix(word.text) for word in nlp(text)])

df2['overview'] = df2['overview'].apply(remove_contradictions)


# #Lemmetization

# from nltk.stem.wordnet import WordNetLemmatizer

# lemmatizer = WordNetLemmatizer()

def Lemmetization(text):

  return " ".join([word.lemma_ for word in nlp(text)])


# #Stemming

# from nltk.stem.porter import PorterStemmer

# ps = PorterStemmer()
# def stemming(text):

#     return " ".join([ps.stem(word.text) for word in nlp(text)])

df2['overview'] = df2['overview'].apply(Lemmetization)




In [1205]:
#Tokenization of overview column

In [1206]:
# from nltk.tokenize import sent_tokenize, word_tokenize

# def token_nlp(string):
#     ls = []
#     # doc1 = nlp(string)
#     doc1 = word_tokenize(string)
#     ls.append(doc1)
#     return ls

In [1207]:
# df2['overview'] = df2['overview'].apply(token_nlp)

In [1208]:
df2.iloc[0, 7]

'22nd century     paraplegic marine dispatch moon pandora unique mission     become torn follow order protect alien civilization'

In [1209]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview
0,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,22nd century paraplegic marine dispatch mo...


In [1210]:
df2['tags'] = df2.sum(axis=1)

In [1211]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview,tags
0,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,22nd century paraplegic marine dispatch mo...,19995


In [1212]:
#Removing spaces between names in cast and crew column

In [1213]:
def remove_spaces(ls):
    for i in range(len(ls)):
        ls[i] = ls[i].replace(' ', '')
    return ls
        # print(ls[i])

In [1214]:
ls1 = df2.iloc[0, 1]

In [1215]:
ls1

['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver']

In [1216]:
remove_spaces(ls1)

['SamWorthington', 'ZoeSaldana', 'SigourneyWeaver']

In [1217]:
df2.drop(columns='tags', axis=1, inplace=True)

In [1218]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,22nd century paraplegic marine dispatch mo...


In [1219]:
df2['cast'] = df2['cast'].apply(remove_spaces)

In [1220]:
df2['director'] = df2['director'].apply(remove_spaces)

In [1221]:
df2['genres'] = df2['genres'].apply(remove_spaces)

In [1222]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, ScienceFiction]","[culture clash, future, space war, space colon...",en,Avatar,22nd century paraplegic marine dispatch mo...


In [1223]:
df2['keywords'] = df2['keywords'].apply(remove_spaces)

In [1224]:
df2['combined'] = df2['cast'] + df2['director'] + df2['genres'] + df2['keywords'] 

In [1225]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview,combined
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...",en,Avatar,22nd century paraplegic marine dispatch mo...,"[SamWorthington, ZoeSaldana, SigourneyWeaver, ..."


In [1226]:
msg = df2.iloc[0, 7]

In [1227]:
df2.iloc[0, 8]

['SamWorthington',
 'ZoeSaldana',
 'SigourneyWeaver',
 'JamesCameron',
 'Action',
 'Adventure',
 'Fantasy',
 'ScienceFiction',
 'cultureclash',
 'future',
 'spacewar',
 'spacecolony',
 'society',
 'spacetravel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alienplanet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'loveaffair',
 'antiwar',
 'powerrelations',
 'mindandsoul',
 '3d']

In [1228]:
def replace_big_spaces(string):
    string = string.replace('  ', '')
    return string

In [1229]:
replace_big_spaces(msg)

'22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization'

In [1230]:
df2['overview'] = df2['overview'].apply(replace_big_spaces)

In [1231]:
df2.head(5)

,id,cast,director,genres,keywords,original_language,original_title,overview,combined
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...",en,Avatar,22nd century paraplegic marine dispatch moon p...,"[SamWorthington, ZoeSaldana, SigourneyWeaver, ..."
1,285,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...",en,Pirates of the Caribbean: At World's End,captain barbossa long believe dead come back l...,"[JohnnyDepp, OrlandoBloom, KeiraKnightley, Gor..."
2,206647,"[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...",en,Spectre,cryptic message bond ’s past send trail uncove...,"[DanielCraig, ChristophWaltz, LéaSeydoux, SamM..."
3,49026,"[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...",en,The Dark Knight Rises,follow death district attorney harvey dent bat...,"[ChristianBale, MichaelCaine, GaryOldman, Chri..."
4,49529,"[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...",en,John Carter,john carter war weary former military captain ...,"[TaylorKitsch, LynnCollins, SamanthaMorton, An..."


In [1232]:
msg = df2.iloc[0, 7]

In [1233]:
msg

'22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization'

In [1234]:
def list_to_string(ls):
    st=""
    
    for i in ls:
        st = ' '.join(ls)
    return st

In [1235]:
msg = df2.iloc[0, 8]

In [1236]:
msg

['SamWorthington',
 'ZoeSaldana',
 'SigourneyWeaver',
 'JamesCameron',
 'Action',
 'Adventure',
 'Fantasy',
 'ScienceFiction',
 'cultureclash',
 'future',
 'spacewar',
 'spacecolony',
 'society',
 'spacetravel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alienplanet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'loveaffair',
 'antiwar',
 'powerrelations',
 'mindandsoul',
 '3d']

In [1237]:
list_to_string(msg)

'SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d'

In [1238]:
df2['combined'] = df2['combined'].apply(list_to_string)

In [1239]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview,combined
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...",en,Avatar,22nd century paraplegic marine dispatch moon p...,SamWorthington ZoeSaldana SigourneyWeaver Jame...


In [1240]:
df2['tags'] = df2['combined'] + df2['overview']

In [1241]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview,combined,tags
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...",en,Avatar,22nd century paraplegic marine dispatch moon p...,SamWorthington ZoeSaldana SigourneyWeaver Jame...,SamWorthington ZoeSaldana SigourneyWeaver Jame...


In [1242]:
df2.iloc[0, 9]

'SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization'

In [1243]:
msg = df2.iloc[0, 5]

In [1244]:
msg

'en'

In [1245]:
msg = ' ' +  msg

In [1246]:
msg

' en'

In [1247]:
def insert_space_at_beginning(string):
    string = ' ' + string
    return string

In [1248]:
df2['original_language'] = df2['original_language'].apply(insert_space_at_beginning)

In [1249]:
df2['tags'] = df2['tags'] + df2['original_language']

In [1250]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview,combined,tags
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...",en,Avatar,22nd century paraplegic marine dispatch moon p...,SamWorthington ZoeSaldana SigourneyWeaver Jame...,SamWorthington ZoeSaldana SigourneyWeaver Jame...


In [1251]:
df2.iloc[0, 9]

'SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization en'

In [1283]:
movies = df2[['id', 'original_title', 'tags']]

In [1284]:
movies.iloc[0,2]

'SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization en'

In [1285]:
movies['original_title'] = movies['original_title'].apply(insert_space_at_beginning)

In [1286]:
movies['tags'] = movies['tags'] + movies['original_title']

In [1287]:
movies.iloc[2, 2]

'DanielCraig ChristophWaltz LéaSeydoux SamMendes Action Adventure Crime spy basedonnovel secretagent sequel mi6 britishsecretservice unitedkingdomcryptic message bond ’s past send trail uncover sinister organization battle political force keep secret service alive bond feel back layer deceit reveal terrible truth behind spectre en Spectre'

In [1288]:
msg = movies.iloc[3, 1]

In [1289]:
def remove_spaces_str(string):
    string = string.replace(' ', '')
    return string

In [1290]:
movies['original_title'] = movies['original_title'].apply(remove_spaces_str)

In [1291]:
movies

,id,original_title,tags
0,19995,Avatar,SamWorthington ZoeSaldana SigourneyWeaver Jame...
1,285,PiratesoftheCaribbean:AtWorld'sEnd,JohnnyDepp OrlandoBloom KeiraKnightley GoreVer...
2,206647,Spectre,DanielCraig ChristophWaltz LéaSeydoux SamMende...
3,49026,TheDarkKnightRises,ChristianBale MichaelCaine GaryOldman Christop...
4,49529,JohnCarter,TaylorKitsch LynnCollins SamanthaMorton Andrew...
...,...,...,...
4798,9367,ElMariachi,CarlosGallardo JaimedeHoyos PeterMarquardt Rob...
4799,72766,Newlyweds,EdwardBurns KerryBishé MarshaDietlein EdwardBu...
4800,231617,"Signed,Sealed,Delivered",EricMabius KristinBooth CrystalLowe ScottSmith...
4801,126186,ShanghaiCalling,DanielHenney ElizaCoupe BillPaxton DanielHsiaa...


In [1292]:
movies.iloc[0,2]

'SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization en Avatar'

In [1293]:
# movies['original_title'] = movies['original_title'].apply(insert_space_at_beginning)

In [1294]:
#Importing the DNN Model

In [1295]:
!pip install gensim

In [1296]:
import gensim

In [1297]:
from gensim.utils import simple_preprocess

In [1298]:
msg = movies.iloc[0, 2]

In [1299]:
msg

'SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization en Avatar'

In [1300]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [1301]:
doc1 = nlp(msg)

In [1302]:
ls = []
for word in doc1:
    ls.append(simple_preprocess(word.text))

In [1303]:
corpus = []
def making_corpus(string):
    msg = nlp(string)
    for word in msg:
        corpus.append(simple_preprocess(word.text))

In [1304]:
ls

[['samworthington'],
 ['zoesaldana'],
 ['sigourneyweaver'],
 ['jamescameron'],
 ['action'],
 ['adventure'],
 ['fantasy'],
 ['sciencefiction'],
 ['cultureclash'],
 ['future'],
 ['spacewar'],
 ['spacecolony'],
 ['society'],
 ['spacetravel'],
 ['futuristic'],
 ['romance'],
 ['space'],
 ['alien'],
 ['tribe'],
 ['alienplanet'],
 ['cgi'],
 ['marine'],
 ['soldier'],
 ['battle'],
 ['loveaffair'],
 ['antiwar'],
 ['powerrelations'],
 ['mindandsoul'],
 ['nd'],
 ['century'],
 ['paraplegic'],
 ['marine'],
 ['dispatch'],
 ['moon'],
 ['pandora'],
 ['unique'],
 ['mission'],
 ['become'],
 ['torn'],
 ['follow'],
 ['order'],
 ['protect'],
 ['alien'],
 ['civilization'],
 ['en'],
 ['avatar']]

In [1305]:
movies['tags'] = movies['tags'].apply(making_corpus)

In [1306]:
corpus

[['samworthington'],
 ['zoesaldana'],
 ['sigourneyweaver'],
 ['jamescameron'],
 ['action'],
 ['adventure'],
 ['fantasy'],
 ['sciencefiction'],
 ['cultureclash'],
 ['future'],
 ['spacewar'],
 ['spacecolony'],
 ['society'],
 ['spacetravel'],
 ['futuristic'],
 ['romance'],
 ['space'],
 ['alien'],
 ['tribe'],
 ['alienplanet'],
 ['cgi'],
 ['marine'],
 ['soldier'],
 ['battle'],
 ['loveaffair'],
 ['antiwar'],
 ['powerrelations'],
 ['mindandsoul'],
 ['nd'],
 ['century'],
 ['paraplegic'],
 ['marine'],
 ['dispatch'],
 ['moon'],
 ['pandora'],
 ['unique'],
 ['mission'],
 ['become'],
 ['torn'],
 ['follow'],
 ['order'],
 ['protect'],
 ['alien'],
 ['civilization'],
 ['en'],
 ['avatar'],
 ['johnnydepp'],
 ['orlandobloom'],
 ['keiraknightley'],
 ['goreverbinski'],
 ['adventure'],
 ['fantasy'],
 ['action'],
 ['ocean'],
 ['drugabuse'],
 ['exoticisland'],
 [],
 ['loveofone', 'slife'],
 ['traitor'],
 ['shipwreck'],
 ['strongwoman'],
 ['ship'],
 ['alliance'],
 ['calypso'],
 ['afterlife'],
 ['fighter'],
 ['p

In [1275]:
#Model Training

In [1276]:
model = gensim.models.Word2Vec(window=1, min_count=5, workers=8, vector_size=200)

In [1277]:
model.build_vocab(corpus)

In [1278]:
model.train(corpus, total_examples=model.corpus_count, epochs=10)

(1682061, 2138840)

In [1279]:
model.wv.most_similar('spiderman')

KeyError: "Key 'spiderman' not present in vocabulary"